In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

#https://github.com/AdrianNunez/Fall-Detection-with-CNNs-and-Optical-Flow

class FallDataset(Dataset):
    def __init__(self, root_dir, sequence_length, transform=None):
        """
        Args:
            root_dir (string): Directory with subdirectories of frames for each clip.
            sequence_length (int): Number of frames to stack as channels.
            transform (callable, optional): Transform to apply to each frame.
        """
        self.root_dir = root_dir
        self.sequence_length = sequence_length
        self.transform = transform
        self.clips = []  # Each entry: (path_to_clip, label)

        # Scan the directory
        for clip_folder in os.listdir(root_dir):
            label = 1 if "FALL" in clip_folder else 0
            self.clips.append((os.path.join(root_dir, clip_folder), label))

    def __len__(self):
        return len(self.clips)

    def __getitem__(self, idx):
        clip_path, label = self.clips[idx]
        frames = []

        # Load each frame in the directory
        for frame_file in sorted(os.listdir(clip_path))[:self.sequence_length]:
            frame_path = os.path.join(clip_path, frame_file)
            frame = Image.open(frame_path).convert("RGB")  # Convert to 3 channels
            if self.transform:
                frame = self.transform(frame)
            frames.append(frame)

        # Stack frames along the channel dimension
        stacked_frames = torch.cat(frames, dim=0)
        return stacked_frames, label


In [3]:
import torchvision.models as models
import torch.nn as nn

#Multi Channel VGG with last 2 FC layers unfrozen for training and 0.9, 0.8 dropout applied to last 2 fc layers

class VGG16MultiChannel(nn.Module):
    def __init__(self, input_channels, num_classes=1):
        super(VGG16MultiChannel, self).__init__()
        
        # Load the pretrained VGG16 model
        self.vgg = models.vgg16(pretrained=True)
        
        # Modify the input layer to accept the specified number of channels
        self.vgg.features[0] = nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1)
        self.vgg.classifier[2] = nn.Dropout(0.9)  # Adjust dropout based on paper
        self.vgg.classifier[5] = nn.Dropout(0.8) # Adjust dropout based on paper
        # Modify the classifier to output a single class for binary classification
        self.vgg.classifier[6] = nn.Linear(4096, num_classes)
        self.sigmoid = nn.Sigmoid()
        
        # Freezing all but last 2 layers based on paper
        for param in self.vgg.parameters():
            param.requires_grad = False  # Freeze all parameters
        # Unfreeze the last two fully connected layers for training
        for param in self.vgg.classifier[3:].parameters():
            param.requires_grad = True  # Unfreeze last 2 layers

    def forward(self, x):
        x = self.vgg(x)
        x = self.sigmoid(x)
        return x


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16MultiChannel(input_channels = 1).to(device)
#View the model
print(model.vgg.features)
print(model.vgg.classifier)

/home/tommy/miniconda3/envs/OFCNN/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tommy/miniconda3/envs/OFCNN/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [11]:
import torch
import torch.optim as optim
import torch.nn as nn

# Parameters
sequence_length = 48  # Number of frames per clip
batch_size = 3
num_epochs = 10
learning_rate = 0.001

# Dataset and DataLoader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = FallDataset(root_dir="../../Data/RFDS/OFCNN/1.0", sequence_length=sequence_length, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16MultiChannel(input_channels = 3*sequence_length).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for frames, labels in train_loader:
        frames, labels = frames.to(device), labels.to(device).float()
        # Forward pass
        optimizer.zero_grad()
        outputs = model(frames)
        outputs = outputs.squeeze()  # Remove extra dimension for binary classification
        print(outputs,labels)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Log training progress
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


tensor([0.4042, 0.6024, 0.6329], grad_fn=<SqueezeBackward0>) tensor([0., 0., 0.])


KeyboardInterrupt: 